In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp,wilcoxon
import math
import random
import pickle
import time


In [6]:
address_main = '/Users/vu/surfdrive/PostDoc/VUMC projrct/Related Matterials/Positive and unlabeled dataset/Code/estimated rho/'
address_data = address_main + 'iris.csv'



# Read Dataset:

In [7]:
df_data = pd.read_csv(address_data,header= None)
df_data.columns = ['x0','x1','x2','x3','text_label']
df_data['real_label']=-1


In [8]:
#df_data.loc[df_data['text_label']=='Iris-setosa','real_label'] = +1      # 1
#df_data.loc[df_data['text_label']=='Iris-versicolor','real_label'] = +1  # 2
df_data.loc[df_data['text_label']=='Iris-virginica','real_label'] = +1    # 3

In [9]:
size_data = len(df_data)
minmax = MinMaxScaler()
df_data[['x0','x1','x2','x3']] = minmax.fit_transform(df_data[['x0','x1','x2','x3']])

In [10]:
# Create Classifiers:


In [24]:
max_counter_ws = 25
max_counter_cs = 10

In [25]:
# Make Tickle for Ws
counter_ws = 0 
Ws = []
while counter_ws < max_counter_ws:
    w = np.random.rand(1,5) *6-3
    df_data['predictione'] = predicted_labels(df_data,w)
    if (np.sum(df_data['predictione'] == +1) != 0) & (np.sum(df_data['predictione'] == -1) != 0)  :
        Ws.append(w)
        counter_ws += 1
with open(address_main + 'Ws'+'.pickle', 'wb') as handle:
    pickle.dump(Ws, handle)

In [29]:
#Read Tickle of Ws
with open(address_main + 'Ws'+'.pickle' , 'rb') as handle:
    Ws = pickle.load(handle)

In [30]:
dic_inversions_F1_= {}
dic_inversions_LL = {}

In [31]:
errors_in_roh = [0.50, 0.75, 0.9, 0.95, 1 , 1.05, 1.1, 1.25 , 1.5, 2]
len(errors_in_roh)

10

In [35]:
c = errors_in_roh[0]
for N_visible in range(1,51):
    rho_real = N_visible / np.sum(df_data['real_label'] == +1)
    rho_estimated = c * rho_real    
    dic_inversions_F1_[rho_real]=[]
    dic_inversions_LL[rho_real]=[]

    for counter_rho in range(0,max_counter_cs):
        print(N_visible,counter_rho )
        df_data['visible_label'] = -1
        temp_visibles = np.random.choice(np.where(df_data['real_label']==+1)[0], size=N_visible, replace=False)
        df_data.loc[temp_visibles,'visible_label'] = +1



        counter_ws = 0
        df_W = pd.DataFrame(columns=['w0','w1','w2','w3','b','F1','F1_','LL'])
        while counter_ws < max_counter_ws:
            w = Ws[counter_ws]
            df_W.loc[counter_ws,['w0','w1','w2','w3','b']] = w
            df_data['predictione'] = predicted_labels(df_data,w)
            df_W.loc[counter_ws,['F1','F1_','LL']] = scores(df=df_data, rho=rho_estimated,writing=False)
            counter_ws += 1
        dic_inversions_F1_[rho_real].append(count_inversions(df_W['F1'],df_W['F1_']))
        dic_inversions_LL[rho_real].append(count_inversions(df_W['F1'],df_W['LL']))


10 0

/Users/vu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/Users/vu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars



10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9


In [58]:
with open(address_main + 'Iric, c=' + ' ' + str(c)+'.pickle', 'wb') as handle:
    pickle.dump([dic_inversions_F1_,dic_inversions_LL], handle)

# FUNCTIONS

#### Calculate Scores

In [1]:
# Linear Function

def g(W,X):
    
    return np.dot(X,w[0,0:-1]) + w[0,-1]

In [2]:
def predicted_labels (df,W):
    N_features = W.shape[1] -1
    for i in range (0 , len(df)):
        Xi = df.iloc[i,0:N_features]
        g_xi = g(W, Xi)
        if g_xi > 0 :
            df.loc[i,'prediction'] = +1
        else:
            df.loc[i,'prediction'] = -1
    return(df['prediction'])

In [3]:
# calculate scores 
def scores(df,rho, writing = False):
    TP = np.sum((df['prediction'] == +1) &  (df ['real_label'] == +1))
    FP = np.sum((df['prediction'] == +1) &  (df ['real_label'] == -1))
    FN = np.sum((df['prediction'] == -1) &  (df ['real_label'] == +1))
    if writing :
        print( 'TP, FP, FN = ', TP, FP, FN)

    pr = TP/(TP+FP)
    re = TP/(TP+FN)
    F_score = 2*pr*re/(pr+re)

    TP_ = np.sum((df['prediction'] == +1) &  (df ['visible_label'] == +1))
    FP_ = np.sum((df['prediction'] == +1) &  (df ['visible_label'] == -1))
    FN_ = np.sum((df['prediction'] == -1) &  (df ['visible_label'] == +1))
    if writing :
        print( 'TP_, FP_, FN_ = ', TP_, FP_, FN_)
    pr_ = TP_/(TP_ + FP_)
    pr_ /= rho
    re_ = TP_/(TP_ + FN_)
    F_score_ = 2*pr_*re_/(pr_+re_)

    L_L_score = re_ **2 / np.sum((df['prediction'] == +1))
    if writing:
        print('np.sum((df[prediction] == +1)) = ', np.sum((df['prediction'] == +1)))
    
    if np.isnan(F_score):
        F_score=0
    if np.isnan(F_score_):
        F_score_=0
    if np.isnan(L_L_score):
        L_L_score=0
    return (F_score , F_score_, L_L_score)

In [4]:

def count_inversions(real_F1s, estimated_scores):
    errors_h = 0

    for i in range(0,len(real_F1s)):
        F1_i = real_F1s[i]
        F1_h_i = estimated_scores[i]
        for j in range(i+1,len(real_F1s)):
            F1_j = real_F1s[j]
            F1_h_j = estimated_scores[j]


            if F1_i > F1_j :
                if F1_h_i < F1_h_j:
                    errors_h += 1

            elif F1_i < F1_j :
                if F1_h_i > F1_h_j:
                    errors_h +=1
                    
    errors_h = errors_h / (len(real_F1s)*(len(real_F1s)-1)/2)

    return(errors_h)